In [ ]:
!pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 73.8 MB/s eta 0:00:00


In [ ]:
import pulp
print(pulp.__version__)  # Para verificar que está instalada correctamente


2.9.0


In [ ]:
import pulp
import pandas as pd

# Datos
supply = {"Planta C": 3, "Planta B": 6, "Planta M": 5, "Planta B2": 4}
demand = {"Cali": 4, "Bogotá": 3, "Medellín": 5, "Barranquilla": 3}
cost = {
    ("Planta C", "Cali"): 1, ("Planta C", "Bogotá"): 4, ("Planta C", "Medellín"): 3, ("Planta C", "Barranquilla"): 6,
    ("Planta B", "Cali"): 3, ("Planta B", "Bogotá"): 1, ("Planta B", "Medellín"): 4, ("Planta B", "Barranquilla"): 5,
    ("Planta M", "Cali"): 4, ("Planta M", "Bogotá"): 4, ("Planta M", "Medellín"): 1, ("Planta M", "Barranquilla"): 4,
    ("Planta B2", "Cali"): 6, ("Planta B2", "Bogotá"): 5, ("Planta B2", "Medellín"): 4, ("Planta B2", "Barranquilla"): 1
}
gen_cost = {"Planta C": 680, "Planta B": 720, "Planta M": 660, "Planta B2": 750}

# Problema de transporte
problem = pulp.LpProblem("Minimizar_Costos_Transporte", pulp.LpMinimize)

# Variables de decisión
routes = pulp.LpVariable.dicts("Route", cost.keys(), lowBound=0, cat="Continuous")

# Función objetivo
problem += pulp.lpSum([routes[(plant, city)] * cost[(plant, city)] for plant, city in cost])

# Restricciones de oferta
for plant in supply:
    problem += pulp.lpSum([routes[(plant, city)] for city in demand]) <= supply[plant], f"Supply_{plant}"

# Restricciones de demanda
for city in demand:
    problem += pulp.lpSum([routes[(plant, city)] for plant in supply]) == demand[city], f"Demand_{city}"

# Resolver el problema
problem.solve()

# Crear tabla de resultados
results = []
for (plant, city), var in routes.items():
    if var.varValue > 0:
        results.append({
            "Planta": plant,
            "Ciudad": city,
            "Energía (GW)": var.varValue,
            "Costo Transporte ($)": var.varValue * cost[(plant, city)],
            "Costo Generación ($)": var.varValue * gen_cost[plant]
        })

# Convertir resultados en DataFrame
df = pd.DataFrame(results)
df["Costo Total ($)"] = df["Costo Transporte ($)"] + df["Costo Generación ($)"]

# Mostrar tabla
print(df)

# Mostrar costos totales
total_cost = df["Costo Total ($)"].sum()
print(f"\nCosto total del suministro: ${total_cost:,.2f}")


      Planta        Ciudad  Energía (GW)  Costo Transporte ($)  \
0   Planta C          Cali           3.0                   3.0   
1   Planta B          Cali           1.0                   3.0   
2   Planta B        Bogotá           3.0                   3.0   
3   Planta M      Medellín           5.0                   5.0   
4  Planta B2  Barranquilla           3.0                   3.0   

   Costo Generación ($)  Costo Total ($)  
0                2040.0           2043.0  
1                 720.0            723.0  
2                2160.0           2163.0  
3                3300.0           3305.0  
4                2250.0           2253.0  

Costo total del suministro: $10,487.00
